<a href="https://colab.research.google.com/github/gukouk176-debug/colab2/blob/main/DataScience_09_ipynb_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第9回講義：分類2
+ 決定木
+ ランダムフォレスト

## 全講義共通初期設定
+ 警告の非表示(実装時は非推奨)
+ numpy pandas小数点以下桁数の表示設定
+ pandas全データ表示設定
+ Google driveへの接続

In [1]:
# ワーニングを非表示にする
# この設定は不都合が見えなくなる為、お勧めしない
# 今回は教育資料用に、出力を簡素化する為に利用する
import warnings
warnings.simplefilter('ignore')

# モジュールの読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 小数点以下桁数の表示設定
np.set_printoptions(precision = 3)
pd.options.display.precision = 3

# pandasの全データ表示設定
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
#google driveに接続
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
#google driveと接続できたかを確認
!ls drive/MyDrive/DataScience

'3(00000).txt'	'3(00004).txt'		       iris.csv
'3(00001).txt'	 breast-cancer-wisconsin.csv   titanic_train.csv
'3(00002).txt'	 example.xlsx		       wine.csv
'3(00003).txt'	 imports-85.csv


# 決定木モデル
アヤメデータに決定木モデルを適用する

In [10]:
#アヤメのデータに決定木モデルを適用する
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import tree

#ライブラリpandasを使ったcsvデータの読み込み,sepで区切り記号を設定
ayame_df = pd.read_csv("/content/drive/MyDrive/DataScience/iris.csv", sep=",")

ayame_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal.length  150 non-null    float64
 1   sepal.width   150 non-null    float64
 2   petal.length  150 non-null    float64
 3   petal.width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [11]:
ayame_df.head()

,sepal.length,sepal.width,petal.length,petal.width,species
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [12]:
#説明変数、目的変数の設定

#X = ayame_df.iloc[:,[0,1,2,3]]
#y = ayame_df.iloc[:,[4]]

X = ayame_df.drop('species', axis=1)
y = ayame_df['species']

print(X.head())
print(y.head())

   sepal.length  sepal.width  petal.length  petal.width
0           5.1          3.5           1.4          0.2
1           4.9          3.0           1.4          0.2
2           4.7          3.2           1.3          0.2
3           4.6          3.1           1.5          0.2
4           5.0          3.6           1.4          0.2
0    Setosa
1    Setosa
2    Setosa
3    Setosa
4    Setosa
Name: species, dtype: object


In [13]:
#テストデータとトレーニングデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None )

#説明変数の標準化
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

#決定木モデルの生成と学習
tree_model = tree.DecisionTreeClassifier(max_depth=3)
tree_model.fit(X_train_std,y_train)

#トレーニングデータによるフィッティング結果とテストデータによる学習モデルの精度の検証
print('正解率（train):  {:.4f}'.format(tree_model.score(X_train_std, y_train)))
print('正解率（test):  {:.4f}'.format(tree_model.score(X_test_std, y_test)))

正解率（train):  0.9810
正解率（test):  0.8889


### 演習1．自動車価格データに対して決定木モデルを適用

In [8]:
#　自動車価格データについて，決定木モデルを用いて prive_over_10K の分類を行う

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

auto_all = pd.read_csv("/content/drive/MyDrive/DataScience/imports-85.csv", header=None)

auto_all.columns = [
    'symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration', 'num-of-doors',
    'body-style', 'drive-wheels', 'engine-location', 'wheel-base', 'length', 'width', 'height',
    'curb-weight', 'engine-type', 'num-of-cylinders', 'engine-size', 'fuel-system', 'bore',
    'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price'
]

features = ['width', 'length', 'engine-size', 'city-mpg']


auto_all = auto_all.replace('?', np.nan)
auto_all['price'] = pd.to_numeric(auto_all['price'], errors='coerce')
auto_all = auto_all.dropna(subset=['price']).copy()

# 目的変数
auto_all['price_over_10K'] = (auto_all['price'] > 10000).astype(int)
auto_df = auto_all[features + ['price_over_10K']].copy()
auto_df[features] = auto_df[features].apply(pd.to_numeric, errors='coerce')
auto_df = auto_df.dropna()

X = auto_df.drop('price_over_10K', axis=1)
y = auto_df['price_over_10K']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0, stratify=y
)

tree_model2 = tree.DecisionTreeClassifier(max_depth=3)
tree_model2.fit(X_train,y_train)

# 精度（accuracy）
print('正解率（train):  {:.4f}'.format(tree_model2.score(X_train, y_train)))
print('正解率（test):   {:.4f}'.format(tree_model2.score(X_test, y_test)))



正解率（train):  0.9286
正解率（test):   0.9180


# ランダムフォレスト

In [14]:
#ランダムフォレストを用いた機械学習
from sklearn.ensemble import RandomForestClassifier

#ランダムフォレストの生成と学習
rf = RandomForestClassifier(max_depth=3)
rf.fit(X_train_std,y_train)

#トレーニングデータによるフィッティング結果とテストデータによる学習モデルの精度の検証
print('正解率（train):  {:.4f}'.format(rf.score(X_train_std, y_train)))
print('正解率（test):  {:.4f}'.format(rf.score(X_test_std, y_test)))

正解率（train):  0.9810
正解率（test):  0.9333


### 演習２．自動車価格データに対してランダムフォレストを適用

In [16]:
# 自動車価格データについて，ランファムフォレストを用いて prive_over_10K の分類を行う．
# 演習１の決定木の結果と比べてみる
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

auto_all = pd.read_csv("/content/drive/MyDrive/DataScience/imports-85.csv", header=None)

auto_all.columns = [
    'symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration', 'num-of-doors',
    'body-style', 'drive-wheels', 'engine-location', 'wheel-base', 'length', 'width', 'height',
    'curb-weight', 'engine-type', 'num-of-cylinders', 'engine-size', 'fuel-system', 'bore',
    'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price'
]

features = ['width', 'length', 'engine-size', 'city-mpg']


auto_all = auto_all.replace('?', np.nan)
auto_all['price'] = pd.to_numeric(auto_all['price'], errors='coerce')
auto_all = auto_all.dropna(subset=['price']).copy()

# 目的変数
auto_all['price_over_10K'] = (auto_all['price'] > 10000).astype(int)
auto_df = auto_all[features + ['price_over_10K']].copy()
auto_df[features] = auto_df[features].apply(pd.to_numeric, errors='coerce')
auto_df = auto_df.dropna()

X = auto_df.drop('price_over_10K', axis=1)
y = auto_df['price_over_10K']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0, stratify=y
)

tree_model2 = RandomForestClassifier(max_depth=3)
tree_model2.fit(X_train,y_train)

# 精度（accuracy）
print('正解率（train):  {:.4f}'.format(tree_model2.score(X_train, y_train)))
print('正解率（test):   {:.4f}'.format(tree_model2.score(X_test, y_test)))


正解率（train):  0.9286
正解率（test):   0.9180
